In [1]:
import pandas as pd
import os

In [11]:
def metadata_filter(filename):
    data = pd.read_excel(filename)

    # Ensure date columns are in datetime format and handle errors
    date_columns = ['Sample Date start H2', 'Sample Date end H2', 
                    'Sample Date start O18', 'Sample Date end O18']

    for col in date_columns:
        data[col] = pd.to_datetime(data[col], errors='coerce')

    # Extract just the date (yyyy-mm-dd), handle NaT values safely
    for col in date_columns:
        data[col] = data[col].apply(lambda x: x.date() if pd.notnull(x) else None)

    # Calculate years of data availability
    data['Years H2'] = (pd.to_datetime(data['Sample Date end H2']) - 
                        pd.to_datetime(data['Sample Date start H2'])).dt.days / 365.25
    data['Years O18'] = (pd.to_datetime(data['Sample Date end O18']) - 
                         pd.to_datetime(data['Sample Date start O18'])).dt.days / 365.25

    # Filter for stations with >10 years of data for both H2 and O18
    filtered_data = data[(data['Years H2'] > 10) & (data['Years O18'] > 10)]
    
    # Drop unnecessary columns
    filtered_data.drop(columns=['Latitude', 'Longitude', 'Altitude', 'Measurand Symbol'], inplace=True)

    return filtered_data

In [14]:
# Initialize an empty DataFrame to collect all filtered data
all_filtered_data = pd.DataFrame()

# Process each file and append the results to the combined DataFrame
for filename in os.listdir():
    if filename.startswith('Metadata_'):
        filtered_metadata_data = metadata_filter(filename)
        all_filtered_data = pd.concat([all_filtered_data, filtered_metadata_data], ignore_index=True)

C:\Users\Sujan\AppData\Local\Temp\ipykernel_19824\3163063891.py:9: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  data[col] = pd.to_datetime(data[col], errors='coerce')
C:\Users\Sujan\AppData\Local\Temp\ipykernel_19824\3163063891.py:9: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  data[col] = pd.to_datetime(data[col], errors='coerce')
C:\Users\Sujan\AppData\Local\Temp\ipykernel_19824\3163063891.py:9: FutureWarning: In a future version of pandas,

In [15]:
filtered_metadata_data_df=pd.DataFrame(all_filtered_data)
filtered_metadata_data_df.to_excel(f"Combined_Metadata.xlsx",index=False)

filtered_metadata_data_df


,Countries/territories,Project Region,Sample Site Name,Sample Date start H2,Sample Date end H2,Sample Date start O18,Sample Date end O18,Years H2,Years O18
0,Africa,Morocco (Fes-Sais),FES SAIS,1994-01-15,2021-12-15,1994-01-15,2021-12-15,27.915127,27.915127
1,Africa,Algeria (Algiers-Univ.),ASSEKREM,1992-05-15,2005-10-18,1965-09-15,2005-10-18,13.426420,40.090349
2,Africa,Tunisia (various sites),TUNIS (CARTHAGE),1968-04-15,2004-12-15,1968-04-15,2004-12-15,36.668036,36.668036
3,Africa,worldwide/closed/noContact,BAMAKO,1962-05-15,2018-09-15,1962-05-15,2018-09-15,56.336756,56.336756
4,Africa,Ascension Island,ASCENSION ISLAND (ATLANTIC O.),1961-12-15,2021-12-15,1961-12-15,2021-12-15,60.000000,60.000000
...,...,...,...,...,...,...,...,...,...
311,Latin America,Chile (various sites),PUERTO MONTT (LA CHAMIZA),1964-05-15,2017-10-15,1964-05-15,2017-10-15,53.418207,53.418207
312,Latin America,Chile (various sites),PUNTA ARENAS,1990-05-15,2017-11-15,1990-05-15,2017-11-15,27.504449,27.504449
313,Latin America,worldwide/closed/noContact,SANTIAGO,1972-08-15,2017-10-15,1972-08-15,2017-10-15,45.166324,45.166324
314,Latin America,Argentina (various sites),RIO CUARTO,2006-10-15,2021-12-15,2006-10-15,2021-12-15,15.167693,15.167693
